In [ ]:
pip install segmentation_models_pytorch

In [ ]:
#加载一些基础的库
import torch
import numpy as np
import os
import torchvision
from tqdm import tqdm #一个实现进度条的库
import random
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import cv2

In [ ]:
class Brightness:
    def __init__(self,brightness_factor):
        self.brightness_factor=brightness_factor
    def __call__(self, img):
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # 通过cv2.cvtColor把图像从BGR转换到HSV
        darker_hsv = img_hsv.copy()
        darker_hsv[:, :, 2] =  self.brightness_factor * darker_hsv[:, :, 2]
        return cv2.cvtColor(darker_hsv, cv2.COLOR_HSV2BGR)   

def image_deal(ori_img):
#     filename = f'/kaggle/input/2dtest/train/train/image/A-{number}.png'
#     imgs = cv2.imread(filename)
    brightness=Brightness(1.2)
    imgs2=brightness(ori_img)
    return imgs2

In [ ]:
from PIL import Image
i=cv2.imread('/kaggle/input/newsegment/segmentation -plus/train/image/6.jpg')
o=Image.open('/kaggle/input/newsegment/segmentation -plus/train/label/6.jpg')
print(type(i))
print(type(o))

In [ ]:
# 图像增强
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256,256))
])

#首先继承Dataset写一个对于数据进行读入和处理的方式
class MyDataset(Dataset):
    def __init__(self,path):
        self.mode=('train' if 'label' in os.listdir(path) else 'test')#表示训练模式
        self.path=path#图片路径
        dirlist=os.listdir(path+'image/')#图片的名称
        self.name=[n for n in dirlist if n[-3:]=='jpg'] #只读取图片
        
    def __len__(self):
        return len(self.name)
    
    def __getitem__(self,index):#获取数据的处理方式
        name=self.name[index]
        #读取原始图片和标签
        if self.mode=='train':#训练模式
          #  print(self.path + 'image/' + name)
           # print(self.path + 'label/' + name)
            ori_img = cv2.imread(self.path + 'image/' + name)  # 原始图片
            l_img = Image.open(self.path + 'label/' + name)  # 标签图片
            lb_img=np.array(l_img)
            #ori_img= image_deal(ori_img)
            #print(type(ori_img))
            #print(type(lb_img))
            ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)  # 转为RGB三通道图
            lb_img = cv2.cvtColor(lb_img, cv2.COLOR_BGR2RGB)  # 掩膜转为灰度图
            lb_img = cv2.cvtColor(lb_img, cv2.COLOR_RGB2GRAY) 
            ori_img = transform1(ori_img)
            lb_img = transform1(lb_img)
            return ori_img, lb_img

        
        if self.mode=='test':#测试模式
            ori_img=cv2.imread(self.path+'image/'+name)#原始图片
            ori_img=cv2.cvtColor(ori_img,cv2.COLOR_BGR2RGB)#转为RGB三通道图
            return transform1(ori_img)

#加载数据集
train_path='/kaggle/input/lovenever/segmentation -plus/train/'
traindata=MyDataset(train_path)
class MyDataset_deal(Dataset):
    def __init__(self,path):
        self.mode=('train' if 'label' in os.listdir(path) else 'test')#表示训练模式
        self.path=path#图片路径
        dirlist=os.listdir(path+'image/')#图片的名称
        self.name=[n for n in dirlist if n[-3:]=='jpg'] #只读取图片
        
    def transform2(self,ori_img,seed):
        torch.manual_seed(seed),
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomRotation(50),  # 随机旋转
        transforms.RandomCrop((256, 512)) ,  # 随机裁剪
        transforms.RandomHorizontalFlip(),  # 随机水平翻转
        transforms.Resize((256, 256))
        ])
        return transform(ori_img)

    def __len__(self):
        return len(self.name)
    
    def __getitem__(self,index):#获取数据的处理方式
        name=self.name[index]
        seed=np.random.randint(0,5200)
        #读取原始图片和标签
        if self.mode=='train':#训练模式
            #print(self.path + 'image/' + name)
            ori_img = cv2.imread(self.path + 'image/' + name)  # 原始图片
            l_img = Image.open(self.path + 'label/' + name)  # 标签图片
            lb_img=np.array(l_img)
            #print(type(ori_img))
            #print(self.path + 'label/' + name)
            #print(type(lb_img))
            #ori_img= image_deal(ori_img)
            ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)  # 转为RGB三通道图
            lb_img = cv2.cvtColor(lb_img, cv2.COLOR_BGR2RGB)  # 掩膜转为灰度图
            lb_img = cv2.cvtColor(lb_img, cv2.COLOR_RGB2GRAY)
            ori_img = self.transform2(ori_img,seed)
            lb_img = self.transform2(lb_img,seed)
            return ori_img, lb_img

#加载数据集
train_path='/kaggle/input/lovenever/segmentation -plus/train/'
traindata_deal=MyDataset_deal(train_path)
from torch.utils.data import ConcatDataset
traindata = ConcatDataset([traindata, traindata_deal])

In [ ]:
import numpy as np
import random
# number = random.choice(range(4000))
#查看图片读取效果
import matplotlib.pyplot as plt
number=np.random.randint(1,20)
# print(type(number))
# print(type(np.random.randint(0,4000))
# print(number)
o_img,l_img=traindata[number]
plt.subplot(1,2,1)
plt.imshow(o_img.permute(1,2, 0))
plt.subplot(1,2,2)
plt.imshow(l_img.permute(1,2, 0))
plt.imshow(o_img[0], cmap='gray')  # 显示原始图像（单通道）
plt.subplot(1, 2, 2)
plt.imshow(l_img[0], cmap='gray')  # 显示标签图像（单通道）
print("原始图片张量的形状:",o_img.shape)
print("标签图片张量的形状:",l_img.shape)#([1, 320, 640]) 其中 1 表示分类类别，我们为2分类任务,类别表示为01

In [ ]:
import torchvision
import torch.nn as nn
import segmentation_models_pytorch as smp
'''
加载经典用于医学图像分割的UNet,encoder_name为模型的backbone
encoder_weigths可选imagenet或者None代表是否加载预训练参数
in_channel为输入图像的通道数
classes为分类数目
'''
model = smp.UnetPlusPlus(
        encoder_name="resnet50",  
        encoder_weights=None,
        in_channels=3,
        classes=1,
    )
##打印模型信息
#print(model) 

In [ ]:
#配置模型超参数
#模型保存的路径
model_path='/kaggle/working/'
#推荐使用gpu进行训练
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#学习率
lr=3e-5
#学习率衰减
weight_decay=1e-3
#批大小
bs=4
#训练轮次
epochs=100

In [ ]:
#训练前准备
from torch.utils.data import DataLoader
#加载模型到gpu或cpu
model.to(device)
#使用Binary CrossEntropy作为损失函数，主要处理二分类问题
# BCEloss=nn.BCELoss()
#加载优化器,使用Adam,主要是炼的快(๑ت๑)
optim=torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
#使用traindata创建dataloader对象
trainloader=DataLoader(traindata,batch_size=bs, shuffle=True, num_workers=1)
#根据赛题评测选用dice_loss，这个是开源代码
def dice_loss(logits, target):
    smooth = 1.
    prob  = torch.sigmoid(logits)
    batch = prob.size(0)
    prob   = prob.view(batch,1,-1)
    target = target.view(batch,1,-1)
    intersection = torch.sum(prob*target, dim=2)
    denominator  = torch.sum(prob, dim=2) + torch.sum(target, dim=2)
    dice = (2*intersection + smooth) / (denominator + smooth)
    dice = torch.mean(dice)
    dice_loss = 1. - dice
    return dice_loss

In [ ]:
#开始炼丹 没有做验证集，各位可以以自己需要去添加
loss_last=99999
best_model_name=''
#记录loss变化
for epoch in range(1,epochs+1):
    for step,(inputs,labels) in tqdm(enumerate(trainloader),desc=f"Epoch {epoch}/{epochs}",
                                       ascii=True, total=len(trainloader)):
        #原始图片和标签
        inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        loss = dice_loss(out, labels)
        # 后向
        optim.zero_grad()
        #梯度反向传播
        loss.backward()
        optim.step()
    #损失小于上一轮则添加
    if loss<loss_last:
        loss_last=loss
        torch.save(model.state_dict(),model_path+'model_epoch{}_loss{}.pth'.format(epoch,loss))
        best_model_name=model_path+'model_epoch{}_loss{}.pth'.format(epoch,loss)
    print(f"\nEpoch: {epoch}/{epochs},DiceLoss:{loss}")

In [ ]:
#加载最优模型
model.load_state_dict(torch.load(best_model_name))
#加载测试集
test_path='/kaggle/input/lovenever/segmentation -plus/test/'
testdata=MyDataset(test_path)
#测试模型的预测效果
x=np.random.randint(0,40)
inputs=testdata[x].to(device)
with torch.no_grad():
    # 模型预测
    t = model(inputs.view(1,3,256,256))
plt.subplot(1,2,1)
plt.imshow(testdata[x].permute(1,2,0))
#对预测的图片采取一定的阈值进行分类
threshold=0.5
t= torch.where(t >=threshold, torch.tensor(255,dtype=torch.float).to(device), t)
t= torch.where(t < threshold, torch.tensor(0,dtype=torch.float).to(device), t)
t=t.cpu().view(1,256,256)
plt.subplot(1,2,2)
plt.imshow(t.permute(1,2,0))

In [ ]:
mkdir output

In [ ]:
from torchvision.utils import save_image
from PIL import Image

img_save_path='/kaggle/working/output/'
for i,inputs in tqdm(enumerate(testdata)):
    #原始图片和标签
    inputs=inputs.reshape(1,3,256,256).to(device)
    # 输出生成的图像
    out = model(inputs.view(1,3,256,256)) # 模型预测
    #对输出的图像进行后处理
    threshold=0.5
    out= torch.where(out >=threshold, torch.tensor(255,dtype=torch.float).to(device),out)
    out= torch.where(out < threshold, torch.tensor(0,dtype=torch.float).to(device),out)
    #保存图像
    out= out.detach().cpu().numpy().reshape(1,256,256)
    #注意保存为1位图提交
    img = Image.fromarray(out[0].astype(np.uint8))
    img = img.convert('1')
    img.save(img_save_path + testdata.name[i])

In [ ]:
#对保存的图像进行打包
import zipfile

def zip_files(file_paths, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in file_paths:
            zipf.write(file)
            
#打包图片
file_paths = [img_save_path+i for i in os.listdir(img_save_path) if i[-3:]=='jpg']
output_path = '/kaggle/working/output.zip'
zip_files(file_paths, output_path)

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import torchvision
import os
"""
In order to reduce the submission file size, our metric uses run-length encoding on the pixel values.
Instead of submitting an exhaustive list of indices for your segmentation, you will submit pairs of values 
that contain a start position and a run length.
E.g. '1 3' implies starting at pixel 1 and running a total of 3 pixels 1,2,3. The competition format requires a space
delimited list of pairs. For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask. 
The metric checks that the pairs are sorted, positive, and the decoded pixel values are not duplicated. 
The pixels are numbered from top to bottom, then left to right: 1 is pixel 1,1, 2 is pixel 2,1, etc. 
The file should contain a header and have the following format: > > img,pixels > 1,1 1 5 1 > 2,1 1 > 3,1 1 > etc.
"""

def get_img_file(image_dir):
    imagelist = []
    namelist = []
    for parent, dirnames, filenames in os.walk(image_dir):
        for filename in filenames:
            if filename.lower().endswith(
                    ('.bmp', '.dib', '.png', '.jpg', '.jpeg', '.pbm', '.pgm', '.ppm', '.tif', '.gif')):
                imagelist.append(os.path.join(parent, filename))
                namelist.append(filename)
        return imagelist, namelist


def turn_to_str(image_list):
    outputs = []
    for image_path in image_list:
        image = Image.open(image_path).convert('L')
        transform = torchvision.transforms.ToTensor()
        image = image.resize((512, 512), Image.Resampling.BILINEAR)
        image = transform(image)
        image[image > 0] = 1
        dots = np.where(image.flatten() == 1)[0]
        run_lengths = []
        prev = -2
        for b in dots:
            if (b > prev + 1):
                run_lengths.extend((b + 1, 0))
            run_lengths[-1] += 1
            prev = b
        output = ' '.join([str(r) for r in run_lengths])
        outputs.append(output)
    return outputs


def save_to_csv(name_list, str_list):
    df = pd.DataFrame(columns=['Id', 'Predicted'])
    #df['Id'] = [i.split('.')[0] for i in name_list]
    df['Id'] = [int(i.replace(".jpg", "")) for i in name_list]
    df['Id'] = name_list
    df['Predicted'] = str_list
    df.head(3)
    df.to_csv('sample_submission.csv', index=None)


if __name__=="__main__":
    image_dir = '/kaggle/working/output'
    image_list, name_list = get_img_file(image_dir)
    str_list = turn_to_str(image_list)
    #name_list = [i for i in range(1,21)]
    #str_list = ["" for i in range(20)]
    #save_to_csv(name_list, str_list)
    df = pd.DataFrame(columns=['Id', 'Predicted'])
    df['Id'] = [i.split('.')[0] for i in name_list]
    
    #df['Id'] = name_list
    df['Predicted'] = str_list
    df.head(3)
    df.to_csv('sample_submission.csv', index=None)
    print(df)